# What is about ?

Notebook for https://www.kaggle.com/alsgroup/end-als

Here we look on the transciptomics part of data and make dimensional reduction and visualization by various methods PCA, UMAP and dozen others. (Transciptomics data - means shows each gene expression for each patient).   53861 genes, 163 patients for the largest dataset.

That is standard procedure working with transciptomics data, sometimes we might be lucky and we can see some clusters corresponding to target variable or having other biological meaning. Unfortunately it is not the case with that data.

We tried many methods, several preprocessings, unfortunately cannot derive useful conclusions for the moment. 


Version 7: first non-draft. 

Versions 1-6: drafts. 



In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
i = 0
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        i += 1
        if i < 5:
            print(os.path.join(dirname, filename))
print('Printed 5 filenames out of ', i)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load Transcriptomics Data (i.e. genes expression matrix)

we also create dataframe df_targets which collects all three targets. In some places it will have NaNs ( NaNs in targets we put for the samples present in one dataset, but not in the other. The first column corresponds to the biggest dataset - it does not have NaNs, since other two - are its subsets).

In [ ]:
list_fn = ['/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/bulbar_vs_limb.csv', 
'/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/median_low_vs_high.csv',
'/kaggle/input/end-als/end-als/transcriptomics-data/DESeq2/ctrl_vs_case.csv']
import time
t0 = time.time()
list_data_df = []
for fn in list_fn:
    d = pd.read_csv(fn)
    list_data_df.append(d)
    print('Loaded:', fn, 'shape:', d.shape)
list_data_names = [ 'bulbar_vs_limb'.replace('_',' '), 'median_low_vs_high'.replace('_',' '),  'ctrl_vs_case'.replace('_',' ') ]

print(time.time()-t0, 'seconds passed')

df = list_data_df[2]
display(df.head(1))
df_targets = df.iloc[:,:2].copy()
df_targets['Order'] = range(df.shape[0])

df_targets = df_targets.set_index(df_targets.columns[0])
for d in list_data_df[:2]:
    d = d.iloc[:,:2]
    d = d.set_index(d.columns[0])
    df_targets = df_targets.join(d, how = 'outer')
df_targets = df_targets.sort_values('Order')
df_targets = df_targets.drop(columns = 'Order')
df_targets.columns = ['ctrl_vs_case'.replace('_',' ') ,  'bulbar_vs_limb'.replace('_',' '), 'median_low_vs_high'.replace('_',' ') ]
df_targets   

In [ ]:
# check: 
(df.iloc[:,1].values != df_targets.iloc[:,0].values ) .sum() # Check 

In [ ]:
df.head(1)

In [ ]:
df_targets.describe()

# Dimensional reduction by PCA 

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA

t0 = time.time()

X = df.iloc[:,2:].values
pca = PCA()# n_components=2)
r = pca.fit_transform(X)
print('Top 10 explained_variance_ratio_', pca.explained_variance_ratio_[:10])
print('Top 10 singular_values_', pca.singular_values_[:10])

print(time.time() - t0,'seconds passed')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(df_targets.columns):
    data_name = col # list_names[i]
    y = df_targets[col].copy()# d.iloc[:,1].values
    y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,3,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(['C9orf72','SOD1','TARDBP','FUS','OPTN','HNRNPA2B1','NIPA1','NEFL' ]):
    data_name = col # list_names[i]
    y = df[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,4,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    if (i+1) % 4 == 0:
        plt.show()
        fig = plt.figure(figsize = (20,8) ); c = 0
        
        
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')



# Dimensional reduction by PCA after cut less variable genes and  StandardScaler

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

print()
print()
print('------------------------------------------------------------------------------------------------------')
print()
print('Keep top variable genes, standard scaler, and only then pca')
print()
print('------------------------------------------------------------------------------------------------------')

t0 = time.time()

X = df.iloc[:,2:].values.copy()

v = np.std(X, axis = 0 )
IX = np.argsort(v)
X = X[:,IX[-10000:]]
print('Keep top',X.shape[1], ' variable genes')

X = scaler.fit_transform(X)

pca = PCA()# n_components=2)
r = pca.fit_transform(X)
print('Top 10 explained_variance_ratio_', pca.explained_variance_ratio_[:10])
print('Top 10 singular_values_', pca.singular_values_[:10])

print(time.time() - t0,'seconds passed')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(df_targets.columns):
    data_name = col # list_names[i]
    y = df_targets[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,3,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(['C9orf72','SOD1','TARDBP','FUS','OPTN','HNRNPA2B1','NIPA1','NEFL' ]):
    data_name = col # list_names[i]
    y = df[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,4,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    if (i+1) % 4 == 0:
        plt.show()
        fig = plt.figure(figsize = (20,8) ); c = 0
        
        
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')



# UMAP

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA

t0 = time.time()

X = df.iloc[:,2:].values
pca = PCA()# n_components=2)
r = pca.fit_transform(X)
r = umap.UMAP(random_state = 42).fit_transform( r[:,:100] )

#print('Top 10 explained_variance_ratio_', pca.explained_variance_ratio_[:10])
#print('Top 10 singular_values_', pca.singular_values_[:10])

print(time.time() - t0,'seconds passed')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(df_targets.columns):
    data_name = col # list_names[i]
    y = df_targets[col].copy()# d.iloc[:,1].values
    y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,3,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.legend()
    
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')

print()
print()
print('------------------------------------------------------------------------------------------------------')
print()
print('Keep top variable genes, standard scaler, and only then pca+UMAP')
print()
print('------------------------------------------------------------------------------------------------------')
t0 = time.time()

X = df.iloc[:,2:].values.copy()

v = np.std(X, axis = 0 )
IX = np.argsort(v)
X = X[:,IX[-10000:]]
print('Keep top',X.shape[1], ' variable genes')

X = scaler.fit_transform(X)

pca = PCA()# n_components=2)
r = pca.fit_transform(X)
r = umap.UMAP(random_state = 42).fit_transform( r[:,:100] )


print(time.time() - t0,'seconds passed')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(df_targets.columns):
    data_name = col # list_names[i]
    y = df_targets[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,3,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('UMAP1')
    plt.ylabel('UMAP2')
    plt.legend()
    
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


# Various dimensional reduction methods from sklearn and not only

In [ ]:
# another analogue of umap which claimed to work a little faster , sometimes produces a little better pictures
# From SkolTech team 
!pip install ncvis
import ncvis
vis = ncvis.NCVis()

!pip install trimap 
import trimap
trimap1 = trimap.TRIMAP()


In [ ]:

# To speed-up reduce dimensions by PCA first

X = df.iloc[:,2:].values.copy()
pca = PCA()# n_components=2)
r = pca.fit_transform(X)
X = r[:,:50]
y = df.iloc[:,1]

import umap 
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter


n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend('')
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0

    else:
        plt.figure(figsize = (20,12))
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)# cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label)# 
        plt.show()

# Various dimensional reduction methods applied after leaving top variable genes and standard scaling

In [ ]:

print()
print('------------------------------------------------------------------------------------------------------')
print()
print('Keep top variable genes, standard scaler, and only then pca+UMAP')
print()
print('------------------------------------------------------------------------------------------------------')
print()
t0 = time.time()

y = df.iloc[:,1]

X = df.iloc[:,2:].values.copy()
v = np.std(X, axis = 0 )
IX = np.argsort(v)
X = X[:,IX[-10000:]]
print('Keep top',X.shape[1], ' variable genes')

X = scaler.fit_transform(X)
print('Standard sclaer applied')

# To speed-up reduce dimensions by PCA first

pca = PCA()# n_components=2)
r = pca.fit_transform(X)
X = r[:,:50]

import umap 
from sklearn import manifold
from sklearn.decomposition import PCA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.ensemble import RandomTreesEmbedding
from sklearn.random_projection import SparseRandomProjection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from sklearn.pipeline import make_pipeline
from sklearn.decomposition import TruncatedSVD


from collections import OrderedDict
from functools import partial
from matplotlib.ticker import NullFormatter


n_neighbors = 10
n_components = 2
# Set-up manifold methods
LLE = partial(manifold.LocallyLinearEmbedding,
              n_neighbors, n_components, eigen_solver='auto')

methods = OrderedDict()
methods['PCA'] = PCA()
methods['umap'] = umap.UMAP(n_components = n_components)
methods['t-SNE'] = manifold.TSNE(n_components=n_components, init='pca', random_state=0)
methods['NCIvis'] = ncvis.NCVis()
methods['trimap'] = trimap.TRIMAP()
methods['ICA'] = FastICA(n_components=n_components,         random_state=0)
methods['FA'] = FactorAnalysis(n_components=n_components, random_state=0)
methods['LLE'] = LLE(method='standard')
methods['Modified LLE'] = LLE(method='modified')
methods['Isomap'] = manifold.Isomap(n_neighbors, n_components)
methods['MDS'] = manifold.MDS(n_components, max_iter=100, n_init=1)
methods['SE'] = manifold.SpectralEmbedding(n_components=n_components,
                                           n_neighbors=n_neighbors)
methods['NMF'] = NMF(n_components=n_components,  init='random', random_state=0) 
methods['RandProj'] = SparseRandomProjection(n_components=n_components, random_state=42)

rand_trees_embed = make_pipeline(RandomTreesEmbedding(n_estimators=200, random_state=0, max_depth=5), TruncatedSVD(n_components=n_components) )
methods['RandTrees'] = rand_trees_embed
methods['LatDirAll'] = LatentDirichletAllocation(n_components=n_components,  random_state=0)
methods['LTSA'] = LLE(method='ltsa') 
methods['Hessian LLE'] = LLE(method='hessian') 

list_fast_methods = ['PCA','umap','FA', 'ICA','NMF','RandProj','RandTrees']
list_slow_methods = ['t-SNE','LLE','Modified LLE','Isomap','MDS','SE','LatDirAll','LTSA','Hessian LLE']

# transformer = NeighborhoodComponentsAnalysis(init='random',  n_components=2, random_state=0) # Cannot be applied since supervised - requires y 
# methods['LinDisA'] = LinearDiscriminantAnalysis(n_components=n_components)# Cannot be applied since supervised - requires y 


# Create figure
fig = plt.figure(figsize=(25, 16))

# Plot results
c = 0
for i, (label, method) in enumerate(methods.items()):
    if 0:
        if label not in  list_fast_methods :
            continue
        
    t0 = time.time()
    try:
        r = method.fit_transform(X.copy())
    except:
        print('Got Exception', label )
        continue 
    t1 = time.time()
    print("%s: %.2g sec" % (label, t1 - t0))
    
    if 1:
        c+=1
        fig.add_subplot(2, 3 , c) 
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)#  cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label )
        plt.legend('')
        if c == 6:
            plt.show()
            fig = plt.figure(figsize=(25, 16))
            c = 0

    else:
        plt.figure(figsize = (20,12))
        sns.scatterplot(x=r[:,0],y=r[:,1] , hue= y)# cell_lines_tissues_top_truncated, style =cell_lines_tissues_top_truncated , s = 250 )# df['drug'] ,  style=df['dose'])
        plt.title(label)# 
        plt.show()

# Choose genes subset used in MoA-kaggle bassed on L1000 Connectivity Map project

Basiclly these about 1000 (772) genes allow more-or-less predict other genes expressions


In [ ]:
# List of genes used in MoA competion and L1000 project
genes772_list = ['AARS1', 'ABCF1', 'ABL1', 'ACAA1', 'ACLY', 'ADAM10', 'ADH5', 'PARP1', 'ADRB2', 'AGL', 'AKT1', 'ALAS1', 'ALDOA', 'ALDOC', 'SLC25A4', 'ANXA7', 'APBB2', 'BIRC2', 'APOE', 'APP', 'FAS', 'RHOA', 'ASAH1', 'ATP1B1', 'ALDH7A1', 'ATP6V0B', 'BAD', 'BAX', 'CCND1', 'BCL2', 'BDH1', 'BID', 'BLMH', 'BMP4', 'BNIP3', 'BPHL', 'BRCA1', 'BUB1B', 'C5', 'DDR1', 'CALU', 'CAPN1', 'CAST', 'CASP2', 'CASP3', 'CASP7', 'CASP10', 'CAT', 'CBLB', 'CBR1', 'CCNA2', 'CCND3', 'CCNF', 'SCARB1', 'CD40', 'CD44', 'CD58', 'ADGRE5', 'CDC20', 'CDC25A', 'CDC25B', 'CDC42', 'CDH3', 'CDK2', 'CDK6', 'CDKN1A', 'CDKN1B', 'CDKN2A', 'CEBPA', 'CEBPD', 'CENPE', 'CETN3', 'CHEK1', 'CIRBP', 'CLTB', 'COL1A1', 'CREB1', 'CRK', 'CRYZ', 'CSK', 'CSNK1A1', 'CSNK1E', 'CSRP1', 'CTNND1', 'CTSD', 'CTSL', 'CYB561', 'DAG1', 'DAXX', 'DCTD', 'DDB2', 'GADD45A', 'DDX10', 'DECR1', 'DFFB', 'DNM1', 'DNMT1', 'DNMT3A', 'DPH2', 'DSG2', 'TSC22D3', 'DUSP3', 'DUSP4', 'TOR1A', 'E2F2', 'ECH1', 'EDN1', 'EGFR', 'EIF4EBP1', 'EIF4G1', 'EIF5', 'ELAVL1', 'EPB41L2', 'EPHA3', 'EPHB2', 'NR2F6', 'ERBB2', 'ERBB3', 'ETFB', 'ETV1', 'EXT1', 'EZH2', 'FAH', 'PTK2B', 'FAT1', 'FDFT1', 'FGFR2', 'FGFR4', 'FHL2', 'FKBP4', 'FOXO3', 'FOS', 'FUT1', 'FYN', 'SLC37A4', 'GAA', 'GABPB1', 'GALE', 'GATA2', 'GFPT1', 'GLRX', 'GNA11', 'GNAI1', 'SFN', 'GPC1', 'GRB10', 'GRN', 'NR3C1', 'CXCL2', 'GSTZ1', 'GTF2A2', 'HSD17B10', 'HADH', 'HIF1A', 'HK1', 'HLA-DMA', 'HMGCR', 'HMGCS1', 'HMOX1', 'HOXA5', 'HOXA10', 'HPRT1', 'HES1', 'DNAJB2', 'HSPA1A', 'HSPA4', 'HSPA8', 'HSPB1', 'DNAJB1', 'ICAM1', 'ID2', 'IFNAR1', 'IGF1R', 'IGF2R', 'IKBKB', 'IL1B', 'IL13RA1', 'ILK', 'INSIG1', 'ITGAE', 'ITGB5', 'JUN', 'KCNK1', 'KIT', 'KTN1', 'STMN1', 'LBR', 'LGALS8', 'LIPA', 'LOXL1', 'LRPAP1', 'SMAD3', 'MAN2B1', 'MAT2A', 'MBNL1', 'MCM3', 'ME2', 'MEF2C', 'MAP3K4', 'MEST', 'MIF', 'FOXO4', 'MMP1', 'MMP2', 'MNAT1', 'MSRA', 'MYBL2', 'MYC', 'GADD45B', 'MYLK', 'MYO10', 'NCK1', 'NFATC3', 'NFATC4', 'NFE2L2', 'NFIL3', 'NFKB2', 'NFKBIA', 'NFKBIE', 'NOS3', 'NOTCH1', 'PNP', 'NPC1', 'NRAS', 'NUCB2', 'NVL', 'ORC1', 'OXA1L', 'OXCT1', 'PAFAH1B1', 'PAFAH1B3', 'SERPINE1', 'PAK1', 'PCBD1', 'PCCB', 'PCK2', 'PCM1', 'PCMT1', 'PCNA', 'PDGFA', 'PFKL', 'PGM1', 'PHKA1', 'PHKB', 'PHKG2', 'PIK3C2B', 'PIK3CA', 'PIN1', 'PLA2G4A', 'PLCB3', 'PLK1', 'PLP2', 'PLS1', 'PLSCR1', 'PMAIP1', 'PMM2', 'POLB', 'POLE2', 'POLR2I', 'PPARD', 'PPARG', 'PPOX', 'PPP2R5A', 'PRKACA', 'PRKCD', 'PRKCH', 'PRKCQ', 'MAPK9', 'PRKX', 'PROS1', 'LGMN', 'PSMB8', 'PSMB10', 'PSMD2', 'PSMD4', 'PSME2', 'PTK2', 'PTPN1', 'PTPN6', 'PTPN12', 'PTPRF', 'PTPRK', 'PXMP2', 'PXN', 'PYCR1', 'PYGL', 'RAB4A', 'RAB27A', 'RAC2', 'RAD9A', 'RAD51C', 'RALA', 'RALB', 'RASA1', 'RB1', 'KDM5A', 'RELB', 'RFC2', 'RFC5', 'RFX5', 'RGS2', 'RHEB', 'RNH1', 'RPA1', 'RPA3', 'MRPL12', 'RPN1', 'RPS6', 'RPS6KA1', 'RSU1', 'RTN2', 'S100A4', 'S100A13', 'SATB1', 'SCP2', 'SDHB', 'SGCB', 'SHB', 'SHC1', 'SKIV2L', 'SKP1', 'SMARCC1', 'SMARCD2', 'SNCA', 'SOX4', 'SPP1', 'SPR', 'SPTAN1', 'SRC', 'STAT1', 'STAT3', 'STAT5B', 'AURKA', 'STK10', 'STX1A', 'STX4', 'STXBP1', 'STXBP2', 'SUV39H1', 'SYK', 'SYPL1', 'TARBP1', 'TBX2', 'TBXA2R', 'TCEA2', 'VPS72', 'TCTA', 'DYNLT3', 'TERT', 'TESK1', 'TFAP2A', 'TFDP1', 'TGFBR2', 'TIAM1', 'TIMP2', 'TJP1', 'TLE1', 'TLR4', 'TSPAN6', 'TSPAN4', 'TOP2A', 'TP53BP2', 'TPD52L2', 'TPM1', 'GFUS', 'TXNRD1', 'UBE2A', 'UGDH', 'NR1H2', 'USP1', 'VDAC1', 'GET1', 'XBP1', 'XPNPEP1', 'ZFP36', 'ZNF131', 'ZMYM2', 'CXCR4', 'IFRD2', 'MAPKAPK3', 'USP7', 'REEP5', 'ST7', 'KAT6A', 'PDHX', 'FOSL1', 'NCOA3', 'NRIP1', 'SMC1A', 'AXIN1', 'CDC45', 'FZD7', 'H2BC21', 'PIP4K2B', 'NCK2', 'DYRK3', 'DUSP11', 'PIK3R3', 'NIPSNAP1', 'ELP1', 'HAT1', 'MAPKAPK5', 'BHLHE40', 'MKNK1', 'CASK', 'AKR7A2', 'RUVBL1', 'PSMG1', 'BECN1', 'MBTPS1', 'EED', 'CTNNAL1', 'RNMT', 'CREG1', 'INPP4B', 'IQGAP1', 'CFLAR', 'ST3GAL5', 'SQSTM1', 'SLC5A6', 'CPNE3', 'TIMELESS', 'P4HA2', 'PLOD3', 'NOL3', 'SLC25A14', 'MPZL1', 'MAP7', 'DNAJA3', 'USP14', 'MTA1', 'PDLIM1', 'SMC3', 'CCNB2', 'CCNE2', 'SYNGR3', 'LPAR2', 'ARHGEF2', 'ZW10', 'AURKB', 'NOLC1', 'UBE2L6', 'MAPKAPK2', 'CYTH1', 'ITGB1BP1', 'BCL7B', 'COPB2', 'ADGRG1', 'TM9SF2', 'MAP4K4', 'HOMER2', 'SH3BP5', 'PIGB', 'PSMF1', 'SPTLC2', 'TBPL1', 'BAG3', 'POLR1C', 'FEZ2', 'HS2ST1', 'IPO13', 'VGLL4', 'UBE3C', 'EDEM1', 'TRAM2', 'CEP57', 'KIAA0100', 'HERPUD1', 'GARRE1', 'USP6NL', 'CCP110', 'MLEC', 'TATDN2', 'MRPL19', 'SCRN1', 'EFCAB14', 'KEAP1', 'MELK', 'PLEKHM1', 'C2CD5', 'KIAA0753', 'C2CD2L', 'TOMM70', 'WASHC5', 'KLHL21', 'FAM20B', 'NCAPD2', 'PAN2', 'LPGAT1', 'KIF14', 'OXSR1', 'MVP', 'DMTF1', 'GNPDA1', 'HDAC6', 'PARP2', 'MAMLD1', 'DNAJB6', 'SMC4', 'ABCC5', 'ABCB6', 'DNM1L', 'TSPAN3', 'KIF20A', 'ARL4C', 'TRAP1', 'MBNL2', 'CEBPZ', 'SLC25A13', 'RBM6', 'TXNDC9', 'TRIM13', 'TRIB1', 'TIMM17B', 'AKAP8', 'STUB1', 'NET1', 'SMNDC1', 'PAK4', 'TNIP1', 'RXYLT1', 'HMG20B', 'MYL9', 'LYPLA1', 'PPIE', 'LRRC41', 'CRTAP', 'VAT1', 'STK25', 'APPBP2', 'CHERP', 'HYOU1', 'RPP38', 'DRAP1', 'PAICS', 'NPRL2', 'YKT6', 'CGRRF1', 'RRAGA', 'GNB5', 'EBP', 'CNPY3', 'YME1L1', 'TCFL5', 'KDM5B', 'ARPP19', 'ZNF274', 'MTHFD2', 'WASF3', 'CLPX', 'PGRMC1', 'MALT1', 'CPSF4', 'BLCAP', 'TCERG1', 'RNPS1', 'TOMM34', 'PDIA5', 'MLLT11', 'EBNA1BP2', 'TMED10', 'ASCC3', 'SLC27A3', 'CCDC85B', 'TLK2', 'RAB31', 'B4GAT1', 'TENT4A', 'UBE2C', 'TOPBP1', 'PRSS23', 'PWP1', 'PKIG', 'CORO1A', 'LSM6', 'PSIP1', 'NISCH', 'CHEK2', 'PRAF2', 'POLG2', 'PNKP', 'ECD', 'DDX42', 'TWF2', 'ATF5', 'MTF2', 'PUF60', 'CLSTN1', 'FOXJ3', 'KHDC4', 'RPIA', 'RAB21', 'SPEN', 'FBXO21', 'WDTC1', 'XPO7', 'TBC1D9B', 'RRP1B', 'MYCBP2', 'CDK19', 'MAST2', 'DCUN1D4', 'ATP11B', 'RRP12', 'SYNE2', 'CAMSAP2', 'ATMIN', 'USP22', 'JADE2', 'ARHGEF12', 'NUDCD3', 'SIRT3', 'SLC35A3', 'ICMT', 'MACF1', 'NNT', 'KLHDC2', 'ACOT9', 'LSM5', 'PLA2G15', 'CEMIP2', 'ZNF318', 'FBXO7', 'BAMBI', 'BACE2', 'MPC2', 'CLIC4', 'C2CD2', 'TIPARP', 'TSKU', 'RNF167', 'LRP10', 'RAI14', 'KIFBP', 'TES', 'PHGDH', 'CHIC2', 'TIMM9', 'ATP2C1', 'TNFRSF21', 'SESN1', 'HTATSF1', 'TMEM97', 'BZW2', 'DNAJC15', 'PACSIN3', 'RBM15B', 'SNX11', 'TIMM22', 'NENF', 'UBQLN2', 'ERO1A', 'DNTTIP2', 'HDGFL3', 'COPS7A', 'NSDHL', 'HEBP1', 'MTERF3', 'AMDHD2', 'ISOC1', 'MRPS16', 'FIS1', 'GOLT1B', 'GLOD4', 'GMNN', 'LAP3', 'NOSIP', 'DERA', 'SCCPDH', 'MRPS2', 'VPS28', 'HSD17B11', 'NUSAP1', 'CD320', 'SNX7', 'ATP6V1D', 'ZNF589', 'PRKAG2', 'UBE2J1', 'EVL', 'HACD3', 'UFM1', 'LSR', 'DHRS7', 'CYCS', 'TERF2IP', 'KCTD5', 'TMCO1', 'EXOSC4', 'DDIT4', 'PAF1', 'SLC35F2', 'ZNF586', 'FBXL12', 'TEX10', 'TXNL4B', 'HERC6', 'PIH1D1', 'PPP2R3C', 'FKBP14', 'CDCA4', 'PLEKHJ1', 'HEATR1', 'UBR7', 'FAIM', 'ADI1', 'ENOSF1', 'CARMIL1', 'ANKRD10', 'CNDP2', 'MINDY1', 'KDM3A', 'PECR', 'EAPP', 'CISD1', 'KLHL9', 'TM9SF3', 'DUSP22', 'COQ8A', 'CIAPIN1', 'ZMIZ1', 'MCOLN1', 'ABHD6', 'TRIB3', 'POLD4', 'SQOR', 'ENOPH1', 'PRUNE1', 'SNX6', 'FASTKD5', 'ELAC2', 'ABHD4', 'RBKS', 'ATG3', 'CIAO3', 'ZDHHC6', 'ACBD3', 'CERK', 'NT5DC2', 'ACD', 'TRAK2', 'METRN', 'TMEM109', 'CCDC86', 'TRAPPC6A', 'CHAC1', 'MBOAT7', 'FSD1', 'TCTN1', 'CHMP6', 'NPEPL1', 'TLCD3A', 'TCEAL4', 'DHDDS', 'COASY', 'TSEN2', 'PRR7', 'ITFG1', 'GRWD1', 'ARID5B', 'TUBB6', 'PSRC1', 'ADO', 'H2BC12', 'UBE3B', 'COG7', 'TBC1D31', 'RPL39L', 'TICAM1', 'SPRED2', 'EML3', 'DIPK1A']

genesALS4_list = ['C9orf72', 'SOD1', 'TARDBP', 'FUS']

print('Intersection 772 and 4',  len(set(genesALS4_list)& set(genes772_list) ) ) 
len(genes772_list)
genes_list = genesALS4_list#  genes772_list

print( len( set( genes_list ) ) )

print ( len(set(df.columns) & set(genes_list) ) )




In [ ]:
len(set(genes772_list) & set(df.columns) ), set(genes772_list) - set(df.columns) 

In [ ]:
import time

import matplotlib.pyplot as plt
import seaborn as sns

import umap
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

print()
print()
print('------------------------------------------------------------------------------------------------------')
print()
print('Keep 771 genes used in MoA kaggle challenge (Connectivity Project) - they more or less allow to predict other genes epxressions ')
print('Apply standard scaler and only then PCA') 
print()
print('------------------------------------------------------------------------------------------------------')

t0 = time.time()

X = df[ set(genes772_list) & set(df.columns)  ].values.copy()

#v = np.std(X, axis = 0 )
#IX = np.argsort(v)
#X = X[:,IX[-10000:]]
#print('Keep top',X.shape[1], ' variable genes')

X = scaler.fit_transform(X)

pca = PCA()# n_components=2)
r = pca.fit_transform(X)
print('Top 10 explained_variance_ratio_', pca.explained_variance_ratio_[:10])
print('Top 10 singular_values_', pca.singular_values_[:10])

print(time.time() - t0,'seconds passed')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(df_targets.columns):
    data_name = col # list_names[i]
    y = df_targets[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,3,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')


fig = plt.figure(figsize = (20,8) ); c = 0
for i,col in enumerate(['C9orf72','SOD1','TARDBP','FUS','OPTN','HNRNPA2B1','NIPA1','NEFL' ]):
    data_name = col # list_names[i]
    y = df[col].copy()# d.iloc[:,1].values
    #y = y.fillna('NAN')
    y = y.values
    
    c += 1; fig.add_subplot(1,4,c)
    
    sns.scatterplot(x = r[:,0], y = r[:,1] , hue = y)
    plt.title(data_name)
    plt.grid()
    plt.xlabel('PCA1')
    plt.ylabel('PCA2')
    plt.legend()
    
    if (i+1) % 4 == 0:
        plt.show()
        fig = plt.figure(figsize = (20,8) ); c = 0
        
        
    #print(time.time() - t0,'seconds passed')

plt.show()
print(time.time() - t0,'seconds passed. Finished.')

